# Approach A Testing and validation

### This notebook tests the model against unseen testing data, and generates performace metrics by means of Precision-recall graph, confusion matrices, classification report and kappa scores

In [ ]:
import numpy as np
import h5py
import tensorflow as tf
from sklearn.metrics import classification_report, precision_recall_curve, f1_score, multilabel_confusion_matrix
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
import seaborn as sns

# High-pass Butterworth filter function with same parameters as HPF in training code.
def highpass_filter(data, cutoff=0.5, fs=400, order=4):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    filtered_data = filtfilt(b, a, data, axis=0)
    return filtered_data

# Load the trained model
model = tf.keras.models.load_model('drive/MyDrive/hpf_model_x/hpf_trained_model.keras') # Replace this path with path to your model

# Load normalisation data
train_mean = np.load('drive/MyDrive/hpf_model_x/hpf_train_mean.npy') # Replace this path with path to your training data mean
train_std = np.load('drive/MyDrive/hpf_model_x/hpf_train_std.npy')# Replace this path with path to your training data standard deviation

# Path to the new combined HDF5 file
test_hdf5_path = 'drive/MyDrive/Combined_Undersampled_normal_14_to_17.h5'

# Initialise lists for test data and labels
test_tracings = []
test_labels = []

# Define arrhythmia columns
arrhythmia_columns = ['1dAVb', 'RBBB', 'LBBB', 'SB', 'ST', 'AF', 'normal_ecg']

# Open the combined HDF5 file and load the test data
with h5py.File(test_hdf5_path, 'r') as f:
    for exam_id in f.keys():
        group = f[exam_id]
        tracing = group['tracing'][:]

        # Apply high-pass filter to the test ECG data
        filtered_tracing = highpass_filter(tracing)

        # Append the filtered tracing and labels
        label = [group.attrs[arrhythmia] for arrhythmia in arrhythmia_columns]
        test_tracings.append(filtered_tracing)
        test_labels.append(label)

# Convert test data to numpy arrays
X_test_balanced = np.array(test_tracings)
y_test_balanced = np.array(test_labels)

# Normalise the test data using the training mean and standard deviation
X_test_normalized = (X_test_balanced - train_mean) / train_std

# Make predictions on the normalised test set
y_pred = model.predict(X_test_normalized)

# For binary classification (normal vs arrhythmia), calculate precision-recall and thresholds
precision = {}
recall = {}
thresholds = {}

for i, label in enumerate(arrhythmia_columns):
    precision[label], recall[label], thresholds[label] = precision_recall_curve(y_test_balanced[:, i], y_pred[:, i])

# Plot precision-recall curve for each class
plt.figure(figsize=(10, 7))
for label in arrhythmia_columns:
    plt.plot(recall[label], precision[label], label=f'Precision-Recall curve for {label}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc="best")
plt.show()

# Find the optimal threshold that maximizes F1 score for each arrhythmia class
optimal_thresholds = {}
for i, label in enumerate(arrhythmia_columns):
    f1_scores = 2 * (precision[label] * recall[label]) / (precision[label] + recall[label])
    optimal_idx = np.argmax(f1_scores)
    optimal_thresholds[label] = thresholds[label][optimal_idx]
    print(f'Optimal threshold for {label}: {optimal_thresholds[label]:.2f} (F1 score: {f1_scores[optimal_idx]:.2f})')

# Now, use the optimal threshold for making predictions
y_pred_optimal = np.zeros_like(y_pred)
for i, label in enumerate(arrhythmia_columns):
    y_pred_optimal[:, i] = (y_pred[:, i] > optimal_thresholds[label]).astype(int)

# Evaluate model performance with the new thresholds
print("Classification Report with optimal thresholds:")
print(classification_report(y_test_balanced, y_pred_optimal, target_names=arrhythmia_columns, zero_division=1))

# Compute confusion matrices for each label
confusion_matrices = multilabel_confusion_matrix(y_test_balanced, y_pred_optimal)

# Plot confusion matrices for each class
fig, axes = plt.subplots(2, 4, figsize=(20, 10))  # Adjust the number of subplots based on your class count
axes = axes.flatten()

for i, (label, cm) in enumerate(zip(arrhythmia_columns, confusion_matrices)):
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[i])
    axes[i].set_title(f'Confusion Matrix for {label}')
    axes[i].set_xlabel('Predicted')
    axes[i].set_ylabel('True')

plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import cohen_kappa_score

for i, label in enumerate(arrhythmia_columns):
    kappa = cohen_kappa_score(y_test_balanced[:, i], y_pred_optimal[:, i])
    print(f'Cohen\'s Kappa Score for {label}: {kappa:.2f}')